In [1]:
#https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00861
import pandas as pd

In [7]:
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
df = pd.read_parquet("nydata_2022.parquet")

In [8]:
df.head()

,ID,NAME,LATITUDE,LONGITUDE,DATE,TAVG,PRCP
0,USW00004781,ISLIP-LI MACARTHUR AP,40.7939,-73.1019,20220102,51.26,0.141732
1,USW00014750,GLENS FALLS AP,43.3386,-73.6103,20220102,31.82,0.118110
2,USW00014750,GLENS FALLS AP,43.3386,-73.6103,20220104,13.28,0.000000
3,USW00014750,GLENS FALLS AP,43.3386,-73.6103,20220105,22.64,0.000000
4,USW00014732,LAGUARDIA AP,40.7794,-73.8803,20220107,33.26,0.330708


In [9]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}
dfst = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [10]:
dfst.columns

Index(['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME', 'GSN FLAG',
       'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [11]:
nyst = dfst[['ID', 'LATITUDE', 'LONGITUDE', 'STATE', 'NAME']][dfst['STATE'].str.match("NY")]

In [12]:
nydf = df[['ID', 'DATE', 'ELEMENT', 'DATA_VALUE']][df['ID'].isin(nyst['ID'])  & df['ELEMENT'].isin(['TAVG', 'PRCP' ])]

KeyError: "['ELEMENT', 'DATA_VALUE'] not in index"

In [22]:
ny = nydf.merge(nyst, on='ID')

In [23]:
nyp = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [25]:
nyp.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyp.columns.to_flat_index())]

In [26]:
nyp['TAVGF'] =  (nyp['TAVG'].astype(float)/10) *(9/5) + 32

In [27]:
nyp['PRCPI'] = nyp['PRCP'].astype(float)/10 * 0.039370 

In [28]:
nyp.to_csv("nydata", index=False)